In [ ]:
from google.colab import drive

drive.mount("/content/drive")

Mounted at /content/drive


In [ ]:
base_dir = '/content/drive/MyDrive/'

In [ ]:
import pandas as pd
import numpy as np
import spacy
import re
from scipy.spatial import distance
import math
import pickle
import string
import pandas as pd
import numpy as np
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize, sent_tokenize
import nltk
import string
import os
import chardet
nltk.download('punkt')
nltk.download('stopwords')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [ ]:
def syllable_count(word):
    word = word.lower()
    count = 0
    vowels = "aeiouy"
    if word[0] in vowels:
        count += 1
    for index in range(1, len(word)):
        if word[index] in vowels and word[index - 1] not in vowels:
            count += 1
            if word[index]=='e':
                count -= 1
    if count == 0:
        count += 1
    return count

def avg_syllable_count(text):
    tokens = sent_tokenize(text)
    l1 = [word for token in tokens for word in token.split()]
    # print(tokens)
    # print(l1)
    # print([syllable_count(word) for word in l1])
    return np.average([syllable_count(word) for word in l1])


#avg word length

def avg_wordlength(text):
    string1 = text.translate(string.punctuation)
    # print(string1)
    tokens = word_tokenize(string1, language='english')
    # print(tokens)
    st = [",", ".", "'", "!", '"', "#", "$", "%", "&", "(", ")", "*", "+", "-", ".", "/", ":", ";", "<", "=", '>', "?",
          "@", "[", "\\", "]", "^", "_", '`', "{", "|", "}", '~', '\t', '\n']
    stop = stopwords.words('english') + st
    words = [word for word in tokens if word not in stop]
    return np.average([len(word) for word in words])


#avg sentence length by number of words

def avg_sentlengthbyword(text):
    tokens = sent_tokenize(text)
    return np.average([len(token.split()) for token in tokens])


def count_functionalwords(text):
    functional_words = """a between in nor some upon
    about both including nothing somebody us
    above but inside of someone used
    after by into off something via
    all can is on such we
    although cos it once than what
    am do its one that whatever
    among down latter onto the when
    an each less opposite their where
    and either like or them whether
    another enough little our these which
    any every lots outside they while
    anybody everybody many over this who
    anyone everyone me own those whoever
    anything everything more past though whom
    are few most per through whose
    around following much plenty till will
    as for must plus to with
    at from my regarding toward within
    be have near same towards without
    because he need several under worth
    before her neither she unless would
    behind him no should unlike yes
    below i nobody since until you
    beside if none so up your
    """

    functional_words = functional_words.split()
    tokens = sent_tokenize(text)
    wo_punc = [text.translate(str.maketrans ('', '', string.punctuation)) for text in tokens]
    l1 = [word for token in wo_punc for word in token.split()]
    count = 0

    for i in l1:
        if i in functional_words:
            count += 1

    return count / len(l1)


#punc count

def punctuation_count_ratio(text):
    st = [",", ".", "'", "!", '"', ";", "?", ":", ";"]
    count = 0
    for i in text:
        if (i in st):
            count = count + 1
    text = text.replace(" ", "")
    # print(text)
    return float(count) / float(len(text))

In [ ]:
!pip install lexicalrichness
from lexicalrichness import LexicalRichness

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 97.8/97.8 kB 1.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for lexicalrichness: filename=lexicalrichness-0.5.1-py3-none-any.whl size=15414 sha256=eec6745d2109822e80c9b5926b8b3f52f956ff6fec9f7350f4b7ac3eaa3fe079
  Stored in directory: /root/.cache/pip/wheels/cd/ba/80/d4dabc1bf242a672ffc00226a2303a7471bb841c0872b2c212
Successfully built lexicalrichness


In [ ]:
#********Readability score********
!pip install readability

import readability
#https://github.com/burrsettles/readability/blob/master/readability.py


#*******Semantic spread*******

!pip install spacy
!python -m spacy download en_core_web_md
!python -m spacy download nl_core_news_lg
!pip install spacy-transformers
nlp = spacy.load("en_core_web_md")
# nlp_trf = spacy.load("en_core_web_trf")
nlp_nl = spacy.load("nl_core_news_lg")

  Preparing metadata (setup.py) ... done
  Created wheel for readability: filename=readability-0.3.1-py3-none-any.whl size=35459 sha256=6111538c42736429a8550b6752085820ac74291f00359f11ab7a15e2e6a021bd
  Stored in directory: /root/.cache/pip/wheels/05/07/4d/2e3a0aaba1713619a403e1a3c56e88a6fc12d753872b98771c
Successfully built readability
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.8/42.8 MB 25.1 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_md')
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 568.1/568.1 MB 1.8 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('nl_core_news_lg')
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 197.9/197.9 kB 1.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 314.0/314.0 kB 6.2 MB/s eta 0:00:00


In [ ]:
def remove_newlines(text):
    cleaned_text = re.sub(r'\n', '', text)
    return cleaned_text


def calculate_semantic_spread(paragraph, lang):
    doc = nlp(paragraph) if lang=="en" else nlp_nl(paragraph)
    sentences = [sent.text for sent in doc.sents]
    # print(sentences)
    spreads = []
    for sentence in sentences:
        sentence_doc = nlp(sentence) if lang=="en" else nlp_nl(sentence)
        # print(sentence_doc)
        vectors = [token.vector for token in sentence_doc]
        # print(vectors)
        similarities = distance.pdist(vectors, metric="cosine")
        # print(similarities)
        spread = np.nanstd(similarities)
        spreads.append(spread)
    return np.nanmean(spreads)

In [ ]:
def style_score(text):
  #lexical features
  avg_syllable_count0 = avg_syllable_count(text)
  avg_wordlength0 = avg_wordlength(text)
  avg_sentlengthbyword0 = avg_sentlengthbyword(text)
  count_functionalwords0 = count_functionalwords(text)
  punctuation_count_ratio0 = punctuation_count_ratio(text)

  #readability scores
  results = readability.getmeasures(text, lang='en')
  fleschreadingease = results['readability grades']['FleschReadingEase']

  #Vocab richness
  lex = LexicalRichness(text)
  yulek = lex.yulek
  maas = lex.Maas
  msttr = lex.msttr(segment_window=15)
  mattr = lex.mattr(window_size=15)

  #Semantic spread
  sem_spread = calculate_semantic_spread(text, 'en')

  vector = [avg_syllable_count0, avg_wordlength0, avg_sentlengthbyword0, count_functionalwords0, punctuation_count_ratio0,

            fleschreadingease,

            yulek, maas, msttr, mattr,

            sem_spread
            ]

  style_dict = {}
  count = 0
  for metric in ['avg_syllable_count', 'avg_wordlength', 'avg_sentlengthbyword', 'count_functionalwords', 'punctuation_count_ratio', 'fleschreadingease', 'yulek', 'maas', 'msttr', 'mattr', 'sem_spread']:
    style_dict[metric] = vector[count]
    count+=1

  return vector, style_dict

In [ ]:
def get_encoding(file):
    with open(file, 'rb') as f:
        result = chardet.detect(f.read())
        # print(result["encoding"])
    return result['encoding']

In [ ]:
categories = ["Poster_Papers", "Rejected_Papers"]

path = os.path.join(base_dir, "ICLR_2018")

all_review_scores = []
count = 0
for cat in categories:
    path1 = os.path.join(path, cat)
    papers = os.listdir(path1)
    papers.sort()
    for paper in papers:
        # print(f"\nCurrent Paper: {paper}")
        path2 = os.path.join(path1, paper)
        docs = os.listdir(path2)
        review_names = [x for x in docs if x[-4:] == '.txt' and not x.__contains__("MetaReview")]
        for review in review_names:
            review_path = os.path.join(path2, review)
            # print(f"Processing Review: {review}")
            with open(review_path, "r", encoding=get_encoding(review_path)) as file:
                content = file.read()
            _, review_scores = style_score(content)
            review_scores["review_name"] = review
            all_review_scores.append(review_scores)
            count += 1
            print(count)

1


/usr/local/lib/python3.10/dist-packages/numpy/lib/nanfunctions.py:1879: RuntimeWarning: Degrees of freedom <= 0 for slice.
  var = nanvar(a, axis=axis, dtype=dtype, out=out, ddof=ddof,


2
3
4
5
6
7


/usr/local/lib/python3.10/dist-packages/numpy/lib/nanfunctions.py:1879: RuntimeWarning: Degrees of freedom <= 0 for slice.
  var = nanvar(a, axis=axis, dtype=dtype, out=out, ddof=ddof,


8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24


...

2411
2412
2413
2414
2415
2416
2417
2418
2419
2420
2421
2422
2423


/usr/local/lib/python3.10/dist-packages/numpy/lib/nanfunctions.py:1879: RuntimeWarning: Degrees of freedom <= 0 for slice.
  var = nanvar(a, axis=axis, dtype=dtype, out=out, ddof=ddof,


2424
2425
2426
2427
2428
2429
2430
2431


/usr/local/lib/python3.10/dist-packages/numpy/lib/nanfunctions.py:1879: RuntimeWarning: Degrees of freedom <= 0 for slice.
  var = nanvar(a, axis=axis, dtype=dtype, out=out, ddof=ddof,


2432
2433
2434
2435
2436
2437
2438
2439
2440
2441


In [ ]:
df = pd.DataFrame.from_dict(all_review_scores)
df.to_csv(os.path.join(base_dir, "Lexical Model", 'review_scores.csv'), index=False)

In [ ]:
print(len(all_review_scores))

2441


In [ ]:
df

,avg_syllable_count,avg_wordlength,avg_sentlengthbyword,count_functionalwords,punctuation_count_ratio,fleschreadingease,yulek,maas,msttr,mattr,sem_spread,review_name
0,1.426506,6.670330,10.375000,0.328502,0.029335,47.084640,138.952091,0.018348,0.923077,0.933508,0.161313,ICLR2018-B12Js_yRb-R2.txt
1,1.518519,7.304598,22.500000,0.335878,0.016160,36.581188,146.504015,0.019886,0.917647,0.916599,0.171135,ICLR2018-B12Js_yRb-R3.txt
2,1.351967,6.259494,16.100000,0.313305,0.032826,63.236203,127.181909,0.018407,0.933333,0.932720,0.175363,ICLR2018-B12Js_yRb-R1.txt
3,1.378472,6.525510,20.571429,0.332143,0.024825,47.908313,94.780508,0.015155,0.962963,0.955725,0.204565,ICLR2018-B13njo1R--R2.txt
4,1.270408,5.668790,14.341463,0.448041,0.024288,52.005353,123.336227,0.019689,0.945614,0.934520,0.190831,ICLR2018-B13njo1R--R1.txt
...,...,...,...,...,...,...,...,...,...,...,...,...
2436,1.547264,6.916115,13.704545,0.250831,0.039892,43.166916,76.518948,0.013844,0.953153,0.955395,0.205214,ICLR2018-ryykVe-0W-R3.txt
2437,1.400545,7.052381,21.588235,0.393443,0.020021,35.749189,119.753086,0.017845,0.950725,0.937380,0.182217,ICLR2018-ryykVe-0W-R2.txt
2438,1.432258,7.313953,22.142857,0.364238,0.022195,53.749784,154.947975,0.019692,0.918519,0.910617,0.191121,ICLR2018-ryzm6BATZ-R3.txt
2439,1.453608,6.830508,19.400000,0.374570,0.022124,31.588185,181.688466,0.019891,0.922807,0.915217,0.171482,ICLR2018-ryzm6BATZ-R1.txt


In [ ]:
df.columns

Index(['avg_syllable_count', 'avg_wordlength', 'avg_sentlengthbyword',
       'count_functionalwords', 'punctuation_count_ratio', 'fleschreadingease',
       'yulek', 'maas', 'msttr', 'mattr', 'sem_spread', 'review_name'],
      dtype='object')